In [1]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [2]:
X,y = load_diabetes(return_X_y=True)

In [3]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [4]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [5]:
lr.coef_

array([  37.90031426, -241.96624835,  542.42575342,  347.70830529,
       -931.46126093,  518.04405547,  163.40353476,  275.31003837,
        736.18909839,   48.67112488])

In [6]:
lr.intercept_

151.3456553477407

In [7]:
y_pred = lr.predict(X_test)
lr.score(X_test,y_test)

0.45260660216173787

In [8]:
r2_score(y_test,y_pred)

0.45260660216173787

In [9]:
for i in range(20):
    print(np.random.randint(0,X_train.shape[0]),end = " ")

293 126 209 193 70 263 338 126 41 240 321 342 288 13 169 174 223 289 239 37 

In [10]:
#Testing data
coef_ = np.ones(X_train.shape[1])
intercept_ = 0
np.dot(X_train[52],coef_) + intercept_

0.4888956794428363

In [11]:
class SDRegressor:
    
    def __init__(self,learning_rate=0.01,epochs=100):
        
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        
    def fit(self,X_train,y_train):
        # init your coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        
        for i in range(self.epochs):
            for j in range(X_train.shape[0]):
                idx = np.random.randint(0,X_train.shape[0])
                
                # This will generate a singel value for Y_hat
                y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_
                
                #derivative of intercept
                intercept_der  = -2 * np.mean(y_train - y_hat)
                
                # derivative for coff
                coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                
                #Appling Gradient Decent
                self.coef_ = self.coef_ - self.lr * coef_der
                
                self.intercept_ = self.intercept_ - self.lr * intercept_der
        
        
        print(self.intercept_,self.coef_)
    
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_
            

In [12]:
sd = SDRegressor(0.01,40)
sd.fit(X_train,y_train)

y_pred = sd.predict(X_test)

r2_score(y_test,y_pred)

155.22347815278556 [  59.95159028  -73.22520813  319.38394495  226.08471139   12.24673881
  -19.78648498 -167.89236881  146.24807517  259.88631602  144.79010396]


0.43549109775261274

- Scostic Gradient Decent is Fast but  as it is taking less nuber of epochs

# Mini Batch GD

- R2 Score = 41%
- intercept = 151

In [13]:
import random
random.sample(range(1,100),10)

[39, 73, 25, 1, 72, 78, 93, 7, 66, 80]

In [14]:
class MBGDRegressor:
    
    def __init__(self,learning_rate=0.01,epochs=100, batch_size = 100):
        
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        
    def fit(self,X_train,y_train):
        # init your coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        
        for i in range(self.epochs):
            for j in range(int(X_train.shape[0]/self.batch_size)):
                idx = random.sample(range(X_train.shape[0]),self.batch_size)
                
                # This will generate a singel value for Y_hat
                y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_
                
                #derivative of intercept
                intercept_der  = -2 * np.mean(y_train[idx] - y_hat)
                
                # derivative for coff
                coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                
                #Appling Gradient Decent
                self.coef_ = self.coef_ - self.lr * coef_der
                
                self.intercept_ = self.intercept_ - self.lr * intercept_der
        
        
        print(self.intercept_,self.coef_)
    
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [15]:
mbd = MBGDRegressor(batch_size=int(X_train.shape[0]/10), learning_rate= 0.01, epochs = 50)

mbd.fit(X_train,y_train)

153.43963053112196 [ 5.12560812e+01 -1.00912703e+02  3.67082096e+02  2.53085305e+02
  3.31982527e-02 -3.51476852e+01 -1.83984772e+02  1.54987936e+02
  2.87203312e+02  1.56346813e+02]


In [16]:
y_pred = mbd.predict(X_test)

r2_score(y_test, y_pred)

0.4497637268703092

## Using inbuild library to solve problem

In [17]:
from sklearn.linear_model import SGDRegressor

sgd = SGDRegressor(learning_rate="constant", eta0=0.1)

In [18]:
#Using  Small trick to convert Schostic GD into Mini Batch GD
batch_size = 35

for i in range(100):
    idx = random.sample(range(X_train.shape[0]),batch_size)
    sgd.partial_fit(X_train[idx],y_train[idx])

In [19]:
y_pred = sgd.predict(X_test)

r2_score(y_test, y_pred)

0.44192922514869615